In [ ]:
from brightway2 import *
from bw2analyzer import *
from bw2regional import *
import csv
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
projects.set_current("LC IMPACT case study")

In [ ]:
sorted(geocollections)

In [ ]:
for act in Database("Fuel comparison"):
    print(act.key, act)

In [ ]:
[x for x in methods if x[0] == 'LC-IMPACT']

In [ ]:
irrigation = [x for x in Database("ecoinvent") if 'irrigation' in x['name']]
crops = [x for x in Database("ecoinvent") 
         if x['name'] in ('sugarcane production', 'maize grain production')]

In [ ]:
activities = [
    ('Fuel comparison', 'sugarcane transport'),
    ('Fuel comparison', 'maize transport'),
    ('Fuel comparison', 'petrol transport'),
]

lcimpact = {
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "Certain", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "All", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "Certain", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "All", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems", "Marginal", "Certain", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "All", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "Certain", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "All", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Aquatic Ecosystems", "Marginal", "All", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Aquatic Ecosystems",  "Marginal", "All", "Infinite"): None, 
 ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-ecoregions-weighted-pop-density',
     'activities': crops,
     'sugarcane': 'world-topo-ecoregions-sugarcane_landuse_intensity',
     'maize': 'world-topo-ecoregions-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'All'): {
     'xt others': 'world-topo-particulate-matter-weighted-pop-density',
     'activities': [],
     'sugarcane': 'world-topo-particulate-matter-sugarcane_landuse_intensity',
     'maize': 'world-topo-particulate-matter-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-particulate-matter-weighted-pop-density',
     'activities': [],
     'sugarcane': 'world-topo-particulate-matter-sugarcane_landuse_intensity',
     'maize': 'world-topo-particulate-matter-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'All'): {
     'xt others': 'world-topo-watersheds-eq-sw-all-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-eq-sw-all-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-eq-sw-all-maize_water_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Human Health', 'Marginal'): {
     'xt others': 'world-topo-watersheds-hh-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-hh-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-hh-maize_water_intensity',
 },
}

In [ ]:
results = []

In [ ]:
HEADERS = ["Name", "Unit", "Ref. product", "Location", "Score (absolute)", "Score (fraction of total)"]

In [ ]:
for method, meta in lcimpact.items():
    print(method)
    results.append(list(method))
    
    fu = ('Fuel comparison', 'petrol transport')
    lca = LCA({fu: 1}, method)
    lca.lci()
    lca.lcia()
    
    results.append([])
    results.append(list(fu))
    results.append(["Site-generic score", lca.score])
    results.append(["Top 10 contributing activities"])
    results.append(HEADERS)
    for score, _, act in ContributionAnalysis().annotated_top_processes(lca)[:10]:
        results.append([act['name'], act['unit'], act['reference product'], act['location'], score, score / lca.score])

    if method[1] != 'Climate Change':        
        xt = ExtensionTablesLCA(
            {fu: 1},
            method,
            xtable=meta['xt others']
        )
        xt.lci()
        xt.lcia()

        results.append([])
        results.append(["Site-dependent score", xt.score])
        results.append(["Top 10 contributing activities"])
        results.append(HEADERS)
        for score, _, act in ContributionAnalysis().annotated_top_processes(xt)[:10]:
            results.append([act['name'], act['unit'], act['reference product'], act['location'], score, score / xt.score])

    for fu, crop in (
            (('Fuel comparison', 'sugarcane transport'), 'sugarcane'),
            (('Fuel comparison', 'maize transport'), 'maize'),
            ):
        
        lca = LCA({fu: 1}, method)
        lca.lci()
        lca.lcia()

        results.append([])
        results.append(list(fu))
        results.append(["Site-generic score", lca.score])
        results.append(["Top 10 contributing activities"])
        results.append(HEADERS)
        for score, _, act in ContributionAnalysis().annotated_top_processes(lca)[:10]:
            results.append([act['name'], act['unit'], act['reference product'], act['location'], score, score / lca.score])

        if method[1] != 'Climate Change':        
            if meta['activities']:
                xt_ag = ExtensionTablesLCA(
                    {fu: 1},
                    method,
                    xtable=meta['sugarcane'],
                    limitations={
                        'activities': meta['activities'],
                    }
                )
                xt_ag.lci()
                xt_ag.lcia()

                xt_others = ExtensionTablesLCA(
                    {fu: 1},
                    method,
                    xtable=meta['xt others'],
                    limitations={
                        'activities': meta['activities'],
                        'activities mode': 'exclude'
                    }
                )
                xt_others.lci()
                xt_others.lcia()

                total = xt_ag.score + xt_others.score

                results.append([])
                results.append(["Site-dependent score", total])
                results.append(["Top 10 contributing activities"])
                results.append(HEADERS)

                contributors = sorted((
                    ContributionAnalysis().annotated_top_processes(xt_ag)[:10] +         
                    ContributionAnalysis().annotated_top_processes(xt_others)[:10]        
                ), reverse=True)[:10]

                for score, _, act in contributors:
                    results.append([act['name'], act['unit'], act['reference product'], act['location'], score, score / total])

            else:
                xt = ExtensionTablesLCA(
                    {fu: 1},
                    method,
                    xtable=meta['xt others']
                )
                xt.lci()
                xt.lcia()

                results.append([])
                results.append(["Site-dependent score", xt.score])
                results.append(["Top 10 contributing activities"])
                results.append(HEADERS)
                for score, _, act in ContributionAnalysis().annotated_top_processes(xt)[:10]:
                    results.append([act['name'], act['unit'], act['reference product'], act['location'], score, score / xt.score])

    results.append([])
    results.append([])

In [ ]:
with open("lcimpact-case-study.csv", "w") as f:
    writer = csv.writer(f)
    for row in results:
        writer.writerow(row)